In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Load the Excel file
file_path = "be0101_invandringkomfland2023_eng (1).xlsx"
df = pd.read_excel(file_path, sheet_name='Table')
df_cleaned = df.drop(columns=['no need', 'no need.1', 'no need.2'])

In [4]:
for col in df_cleaned.columns[1:]:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce').fillna(0).astype(int)

In [5]:
df_cleaned['Soviet Union and Unknown country'] = df_cleaned['Soviet Union'] + df_cleaned['Unknown country']
df_cleaned = df_cleaned.drop(columns=['Soviet Union', 'Unknown country'])

In [6]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 24 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   County                                         19 non-null     object
 1   Total                                          19 non-null     int32 
 2   Sweden                                         19 non-null     int32 
 3   Denmark                                        19 non-null     int32 
 4   Finland                                        19 non-null     int32 
 5   Iceland                                        19 non-null     int32 
 6   Norway                                         19 non-null     int32 
 7   EU except the Nordic countries                 19 non-null     int32 
 8   Europe except the EU and the Nordic countries  19 non-null     int32 
 9   Türkiye                                        19 non-null     int3

In [7]:
unique_counties = df_cleaned['County'].unique()
unique_counties

array(['Stockholm county', 'Uppsala county', 'Östergötland county',
       'Jönköping county', 'Kronoberg county', 'Kalmar county',
       'Gotland county', 'Blekinge county', 'Skåne county',
       'Halland county', 'Västra Götaland county', 'Värmland county',
       'Västmanland county', 'Dalarna county', 'Gävleborg county',
       'Västernorrland county', 'Jämtland county', 'Västerbotten county',
       'Norrbotten county'], dtype=object)

In [206]:
country_columns = df_cleaned.columns[2:]  # İlk iki kolonu atla (County ve Total)
print(country_columns)

In [207]:
colors = plt.cm.Paired.colors

for county in unique_counties:
    county_data = df_cleaned[df_cleaned['County'] == county].drop(columns=['County'])
    total_immigrants = county_data['Total'].sum()
    county_country_distribution = county_data.drop(columns=['Total']).sum().sort_values(ascending=False)
    
    # Calculate percentages
    county_country_percentage = (county_country_distribution / total_immigrants) * 100
    
    fig, ax = plt.subplots(figsize=(12, 8))
    bars = ax.bar(county_country_distribution.index, county_country_distribution.values, 
                  color=[colors[i % len(colors)] for i in range(len(county_country_distribution))])

    ax.set_title(f'Country Distribution in {county}')
    ax.set_xlabel('Country')
    ax.set_ylabel('Number of Immigrants')

    # Add total number of immigrants as a text in the plot
    ax.text(len(county_country_distribution) / 2, max(county_country_distribution.values) * 0.9, f'Total: {total_immigrants}', 
            ha='center', va='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

    # Adding value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}', 
                    xy=(bar.get_x() + bar.get_width() / 2, height), 
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points", 
                    ha='center', va='bottom')

    plt.xticks(rotation=90)

    # Create the legend
    handles = [plt.Line2D([0], [0], color=colors[i % len(colors)], lw=4) for i in range(len(county_country_distribution))]
    labels = [f'{country}: {county_country_percentage[country]:.1f}%' for country in county_country_distribution.index]
    plt.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()